In [1]:
!pip install keras

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple


In [2]:
import pandas as pd
import numpy as np

In [3]:
train=pd.read_csv("train_all.csv")
test=pd.read_csv("test_all.csv")
all=pd.concat([train,test],ignore_index=True)

In [4]:
item=all["user_items"].values.tolist()

In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
docs = item

# 只考虑最常见的词
max_words = 3000
# 统一的序列化长度
# 大于这个长度截断，小于这个长度用0补全
maxlen = 10
# 嵌入的维度
embedding_dim = 32

# 分词
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(docs)
# 字典
word_index = tokenizer.word_index

# 序列化
sequences = tokenizer.texts_to_sequences(docs)
# 统一序列长度
data = pad_sequences(sequences, maxlen=maxlen)

# Embedding模型
model = Sequential()
# Embedding至少需要max_wrods和embedding_dim两个参数
model.add(Embedding(max_words, embedding_dim, input_length=maxlen, name='embedding'))
model.compile('rmsprop', 'mse')

out = model.predict(data)
# print(out)
# print(out.shape)
# # 查看权重
# layer = model.get_layer('embedding')
# print(layer.get_weights())

embedding_list=[]
for i in out:
    temp=sum(i)/10
    temp=temp.tolist()
    embedding_list.append(temp)

Using TensorFlow backend.


In [6]:
train_list=embedding_list[0:900000]
test_list=embedding_list[900000:1900000]

In [7]:
# a=pd.DataFrame(embedding_list)

In [8]:
# train.drop(["user_items"],axis=1,inplace=True)

In [9]:
# train_em=pd.concat([train,a],axis=1)

In [10]:
# train_em.to_csv("train_em.csv")

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
%matplotlib inline

In [20]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [22]:

max_words = 1000
max_len = 150
model = RNN()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 150)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [23]:
from sklearn.model_selection import train_test_split
target=np.array(train["gender"])#性别建模
X_train,X_test,y_train,y_test=train_test_split(train_list,target,test_size=0.03)

In [ ]:
model.fit(X_train,y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

In [ ]:
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
preds = model.predict(test_list)
gender_pred=np.rint(preds)

In [ ]:
submission=pd.DataFrame(test["user_id"])

In [ ]:
submission["predicted_age"]=age_pred
submission["predicted_gender"]=gender_pred

In [ ]:
submission.to_csv("submission_gender.csv",index=False)

In [ ]:
submission